In [9]:
import pandas as pd
import urllib
import os
import pickle
import rassp
import torch
from rassp import netutil
from rdkit import Chem

In [7]:
# XXX: needed by pickle.load of rassp models

from rassp import msutil,model
import sys
sys.modules['msutil'] = msutil
sys.modules['model'] = model

In [2]:
s = pd.read_parquet('rassp-public/sample_data/smallmols_cfm_pred_public_sample.parquet')

In [7]:
s.iloc[0].spect

array([array([15.02292522,  0.91388646]),
       array([25.00727602,  0.96256718]),
       array([27.02292522,  3.49654964]),
       array([29.03857442,  2.88757419]),
       array([39.02292522,  0.92641582]),
       array([41.03857442,  3.63968496]),
       array([42.04639902,  2.29737418]),
       array([43.05422362,  8.79653005]),
       array([43.97152142,  2.10668418]),
       array([44.06204822,  1.32544061]),
       array([44.97934602,  2.70170413]),
       array([45.06987282,  1.27590546]),
       array([55.05422362,  2.72504282]),
       array([58.98241922,  4.22013733]),
       array([59.99024382,  2.07733632]),
       array([69.06987282,  2.20343483]),
       array([70.07769742,  1.42581885]),
       array([71.08552202,  9.44191393]),
       array([84.99806842,  1.50804528]),
       array([86.00589302, 18.14332272]),
       array([87.01371762,  1.56864662]),
       array([88.02154222,  1.06527212]),
       array([114.0371914 ,   2.45328396]),
       array([129.0606652 ,   2.

In [8]:
s.iloc[0].spect_sparse

array([array([15.        ,  0.91388646]),
       array([25.        ,  0.96256718]),
       array([27.        ,  3.49654964]),
       array([29.        ,  2.88757419]),
       array([39.        ,  0.92641582]),
       array([41.        ,  3.63968496]),
       array([42.        ,  2.29737418]),
       array([43.        ,  8.79653005]),
       array([44.        ,  2.10668418]),
       array([44.        ,  1.32544061]),
       array([45.        ,  2.70170413]),
       array([45.        ,  1.27590546]),
       array([55.        ,  2.72504282]),
       array([59.        ,  4.22013733]),
       array([60.        ,  2.07733632]),
       array([69.        ,  2.20343483]),
       array([70.        ,  1.42581885]),
       array([71.        ,  9.44191393]),
       array([85.        ,  1.50804528]),
       array([86.        , 18.14332272]),
       array([87.        ,  1.56864662]),
       array([88.        ,  1.06527212]),
       array([114.        ,   2.45328396]),
       array([129.        ,   2.

In [9]:
s.iloc[0]

mol_id                                              NIST2014_10006
inchi                InChI=1S/C6H11NS/c1-5(2)6(3)7-4-8/h5-6H,1-3H3
smiles                                              CC(C)C(C)N=C=S
rdmol            b'\xef\xbe\xad\xde\x00\x00\x00\x00\x0c\x00\x00...
cv_id                                                            7
morgan4_crc32                                           3981836209
spect            [[15.02292522, 0.9138864566], [25.00727602, 0....
spect_sparse     [[15.0, 0.9138864566], [25.0, 0.9625671785], [...
inchi_key                              YAVRRVJBEIYPRH-UHFFFAOYSA-N
Name: 0, dtype: object

### TODO 

Z toho pouziva RASSP jen spect a rdmol
rdmol je serializovany rdkit.Chem.Mol

to budeme umet z MSP vyrobit

In [14]:
base='https://people.cs.uchicago.edu/~ericj/rassp/'
files=[ 'formulanet_best_candidate_pcsim_pretrain.nist-fromscratch-3x9x128.35790555.00000740.model',    
'formulanet_best_candidate_pcsim_pretrain.nist-fromscratch-3x9x128.35790555.meta',
'subsetnet_best_candidate_nist_fromscratch.nist-fromscratch-test01-old-1x2048.36688199.00001000.model', 
'subsetnet_best_candidate_nist_fromscratch.nist-fromscratch-test01-old-1x2048.36688199.meta',   
'subsetnet_best_candidate_nist_posttrain_cluster.cluster-nist-posttrain.35946017.00000480.model',
'subsetnet_best_candidate_nist_posttrain_cluster.cluster-nist-posttrain.35946017.meta']

for f in files:
    urllib.request.urlretrieve(f'{base}/{f}',f)

In [8]:
# from original run_rassp.py

use_gpu=True
model_dir='.'

MODELS = {
    'FormulaNet': {
        'checkpoint': os.path.join(model_dir, 'formulanet_best_candidate_pcsim_pretrain.nist-fromscratch-3x9x128.35790555.00000740.model'),
        'meta': os.path.join(model_dir, 'formulanet_best_candidate_pcsim_pretrain.nist-fromscratch-3x9x128.35790555.meta'),

        # NOTE(2023-01-24): FN currently has a reduced set of valid mol constraints bc this is what we trained with
        # and there is currently an outstanding bug where changing N_ATOMS here breaks FN predictions in subtle ways.
        'override_constraints': {
            'n_atom': 48,
            # 'n_formula': 4096,
            'n_formula': 32768,
            # 'n_subset': 12288,
            'n_subset': 49152,
        },
    },
    'SubsetNet': {
        'checkpoint': os.path.join(model_dir, 'subsetnet_best_candidate_nist_fromscratch.nist-fromscratch-test01-old-1x2048.36688199.00001000.model'),
        'meta': os.path.join(model_dir, 'subsetnet_best_candidate_nist_fromscratch.nist-fromscratch-test01-old-1x2048.36688199.meta'),

        # NOTE: this set of params works for SN GPU inference on a single RTX 2080 Ti (12GB VRAM)
        # despite SN being trained on <= 48 atoms like FN, it also scales well to mols up to <= 64 atoms (tested)
        'override_constraints': {
            'n_atom': 64,
            'n_formula': 32768,
            'n_subset': 49152,
        },
    },
}

model_name='FormulaNet'
meta_path = MODELS[model_name]['meta']
ckpt_path = MODELS[model_name]['checkpoint']
override_constraints = MODELS[model_name]['override_constraints']
meta = pickle.load(open(meta_path, 'rb'))

feat_config = meta['featurize_config']
for key, val in override_constraints.items():
    if key == 'n_atom':
        feat_config['MAX_N'] = val
    if key == 'n_formula':
        feat_config['explicit_formulae_config']['max_formulae'] = val
    if key == 'n_subset':
        feat_config['vert_subset_samples_n'] = val

if use_gpu and not torch.cuda.is_available():
    use_gpu = False

predictor = netutil.PredModel(
    meta_path,
    ckpt_path,
    USE_CUDA=use_gpu,
    data_parallel=False,
    featurize_config_update=feat_config,
)


In [10]:
smiles = ['C1=CNN=C1','CC(C)C#N','CCC(=C)C','CC[N+](=O)[O-]']

valid_mols = [ Chem.AddHs(Chem.MolFromSmiles(s)) for s in smiles ]

In [12]:
predictions = predictor.pred(
    valid_mols,
    progress_bar=True,
    normalize_pred=True,
    output_hist_bins=True,
    batch_size=4, # XXX
    dataloader_config={
        'pin_memory': False,
        'num_workers': 0, # XXX
        'persistent_workers': False,
    },
    benchmark_dataloader=False,
)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  4.78it/s]


In [20]:
predictions['pred_binned'][0]

array([[1.2000000e+01, 1.5725836e-05],
       [1.3000000e+01, 2.5566214e-05],
       [1.4000000e+01, 8.4828343e-06],
       [1.5000000e+01, 1.5869243e-06],
       [1.6000000e+01, 2.2642298e-05],
       [2.4000000e+01, 6.1679937e-05],
       [2.5000000e+01, 1.6761186e-04],
       [2.6000000e+01, 1.2685906e-03],
       [2.7000000e+01, 1.3037211e-03],
       [2.8000000e+01, 1.5958099e-02],
       [2.9000000e+01, 2.8420868e-04],
       [3.0000000e+01, 1.6846336e-05],
       [3.1000000e+01, 5.4366119e-06],
       [3.2000000e+01, 3.5220235e-06],
       [3.6000000e+01, 3.0995213e-04],
       [3.7000000e+01, 8.7629817e-04],
       [3.8000000e+01, 7.5590909e-03],
       [3.9000000e+01, 2.2764167e-02],
       [4.0000000e+01, 9.9940322e-02],
       [4.1000000e+01, 1.6757661e-01],
       [4.2000000e+01, 4.8595341e-03],
       [4.3000000e+01, 9.6873526e-05],
       [4.4000000e+01, 1.6739970e-05],
       [5.0000000e+01, 1.6946797e-03],
       [5.1000000e+01, 1.1282132e-02],
       [5.2000000e+01, 3.